# Worksheet 08

Name: Mao Mao
UID: U02043894

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [10]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.3872874911119375, 5.602240697211914, 5.606244552519783, 6.960422588952641, 3.918946670418345, 5.037117001411848, 4.390436145995281, 5.348037469330129, 4.132983533857102, 7.467672043383739]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [11]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[8.683511913428664, 6.834150183375982, 7.246826622706217, 9.228791689664456, 10.010227996795855, 8.300576559880598, 8.190242726889393, 7.998371841866975, 8.650724230450386, 7.635000050806957]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [12]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[7.467672043383739, 4.132983533857102, 5.348037469330129, 4.390436145995281, 5.037117001411848, 3.918946670418345, 7.635000050806957, 6.960422588952641, 8.650724230450386, 7.998371841866975]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Mean and Variance, and Probability of x coming from specie i

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [13]:
kmeans = KMeans(2, init='k-means++', n_init='auto').fit(X=np.array(data).reshape(-1, 1))

In [14]:
s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [len(s1) / (len(s1) + len(s2)), len(s2) / (len(s1) + len(s2))]
mean = [sum(s1) / len(s1), sum(s2) / len(s2)]
var = [sum(map(lambda x: (x - mean[0]) ** 2, s1)) / len(s1), sum(map(lambda x: (x - mean[1]) ** 2, s2)) / len(s2)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))
print("")

[7.467672043383739, 7.635000050806957, 6.960422588952641, 8.650724230450386, 7.998371841866975]
[4.132983533857102, 5.348037469330129, 4.390436145995281, 5.037117001411848, 3.918946670418345]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 7.74243815109214,  mean_2 = 4.565504164202541
var_1 = 0.3178146709592148,  var_2 = 0.2941073082592907



mean_1 = 7.74243815109214,  
mean_2 = 4.565504164202541, 
var_1 = 0.3178146709592148,  
var_2 = 0.2941073082592907

assigned mean_1 to c2, and mean_2 to c1.
mean_1 is 8, var is 1. mean_2 is 5, var is 1. The variance for mean_1 differs, but the mean is pretty close. Since we are taking 4 data points from c1 and 6 data points from c2, the predicted probability becomes the true value .

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [15]:
from scipy.stats import norm

prob_s0_x = []  # P(S_0 | X_i)
prob_s1_x = []  # P(S_1 | X_i)
prob_x = []  # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        # prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append((pdf_i[0] * prob_s[0]) / prob_x)
    prob_s1_x.append((pdf_i[1] * prob_s[1]) / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  7.467672043383739
probability of observing that point if it came from cluster 0 =  0.863835015442807
probability of observing that point if it came from cluster 1 =  9.735881688552135e-22
point =  4.132983533857102
probability of observing that point if it came from cluster 0 =  1.230982855153613e-28
probability of observing that point if it came from cluster 1 =  0.46001681088378804
point =  5.348037469330129
probability of observing that point if it came from cluster 0 =  5.934206308094679e-13
probability of observing that point if it came from cluster 1 =  0.03936775505461805
point =  4.390436145995281
probability of observing that point if it came from cluster 0 =  8.775967695331607e-25
probability of observing that point if it came from cluster 1 =  1.1362228460849801
point =  5.037117001411848
probability of observing that point if it came from cluster 0 =  2.3150322706195873e-16
probability of observing that point if it came from cluster 1 =  0.37501163106216384
point =

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [16]:
prob_c = [sum(prob_s0_x) / len(prob_s0_x), sum(prob_s1_x) / len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),
        sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),
       sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 7.742438151084935,  mean_2 = 4.565504164200224
var_1 = 0.3178146709755364,  var_2 = 0.2941073082584281


Cluster 1 has a higher mean and less variant.
Cluster 2 has a lower mean value and more variant.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [17]:
prob_s0_x = []  # P(S_0 | X_i)
prob_s1_x = []  # P(S_1 | X_i)
prob_x = []  # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        # prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append((pdf_i[0] * prob_s[0]) / prob_x)
    prob_s1_x.append((pdf_i[1] * prob_s[1]) / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

point =  7.467672043383739
probability of observing that point if it came from cluster 0 =  0.8638350154485336
probability of observing that point if it came from cluster 1 =  9.735881685043294e-22
point =  4.132983533857102
probability of observing that point if it came from cluster 0 =  1.2309828635614297e-28
probability of observing that point if it came from cluster 1 =  0.4600168108875495
point =  5.348037469330129
probability of observing that point if it came from cluster 0 =  5.934206326101518e-13
probability of observing that point if it came from cluster 1 =  0.03936775505309083
point =  4.390436145995281
probability of observing that point if it came from cluster 0 =  8.77596774711475e-25
probability of observing that point if it came from cluster 1 =  1.1362228460924606
point =  5.037117001411848
probability of observing that point if it came from cluster 0 =  2.31503227956207e-16
probability of observing that point if it came from cluster 1 =  0.3750116310556975
point =  3

For the 6.394 point, it seems to have changed the assignments. Now the probability of coming from s2 is 0.869, and previously it was 0.354. It seems to have changed the preferred assignment.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [18]:
assignments = [[], []]
for x in zip(data, prob_s0_x):
    p = x[1]
    d = x[0]
    rand = random.random()
    if rand <= p:
        assignments[0].append(d)
    else:
        assignments[1].append(d)
print(f"points in cluster 0 : {assignments[0]}\npoints in cluster 1 : {assignments[1]}")

points in cluster 0 : [7.467672043383739, 7.635000050806957, 6.960422588952641, 8.650724230450386, 7.998371841866975]
points in cluster 1 : [4.132983533857102, 5.348037469330129, 4.390436145995281, 5.037117001411848, 3.918946670418345]
